# Train/test cycles example using PaiNN

In [1]:
import os
import tqdm

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import schnetpack as spk
import schnetpack.representation as rep
import schnetpack.task as task
import schnetpack.transform as trn

import torch
import torchmetrics
from torch.optim.lr_scheduler import ReduceLROnPlateau

from nablaDFT.dataset import NablaDFT
from nablaDFT.painn.train_painn import AtomisticTaskFixed, seed_everything

In [2]:
dataset_name = 'dataset_train_2k'  # Name of the training dataset
datapath = 'database'              # Path to the selected dataset
logspath = 'logs'                  # Path to log files
nepochs = 200                        # Number of epochs to train for
seed = 1799                        # Random seed number for reproducibility
batch_size = 100                   # Size of each batch for training
n_interactions = 6                 # Number of interactions to consider between atoms
n_atom_basis = 128                 # Number of basis functions for atoms in the representation
n_rbf = 20                         # Number of radial basis functions in the representation
cutoff = 5.0                       # Cutoff distance (in Bohr) for computing interactions
devices = 1                        # Number of GPU/TPU/CPU devices to use for training

## Downloading dataset

In [3]:
seed_everything(seed)
workpath = logspath

if not os.path.exists(workpath):
    os.makedirs(workpath)

data = NablaDFT(
    "ASE",
    dataset_name,
    datapath=datapath,
    data_workdir=workpath,
    batch_size=batch_size,
    num_workers=4,
    transforms=[
        trn.ASENeighborList(cutoff=cutoff),
        trn.RemoveOffsets("energy", remove_mean=True, remove_atomrefs=False),
        trn.CastTo32(),
    ],
    split_file=os.path.join(workpath, "split.npz"))

## Initializing training procedure and starting training

In [4]:
pairwise_distance = spk.atomistic.PairwiseDistances()
radial_basis = spk.nn.radial.GaussianRBF(
    n_rbf=n_rbf,
    cutoff=cutoff
)
cutoff_fn = spk.nn.cutoff.CosineCutoff(cutoff)
representation = rep.PaiNN(
    n_interactions=n_interactions,
    n_atom_basis=n_atom_basis,
    radial_basis=radial_basis,
    cutoff_fn=cutoff_fn
)
pred_energy = spk.atomistic.Atomwise(
    n_in=representation.n_atom_basis,
    output_key="energy"
)
postprocessors = [
    trn.CastTo64(),
    trn.AddOffsets("energy", add_mean=True)
]
nnpot = spk.model.NeuralNetworkPotential(
    representation=representation,
    input_modules=[pairwise_distance],
    output_modules=[pred_energy],
    postprocessors=postprocessors
)
output_energy = spk.task.ModelOutput(
    name="energy",
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1,
    metrics={"MAE": torchmetrics.MeanAbsoluteError()}
)

scheduler_args = {
    "factor": 0.8,
    "patience": 10,
    "min_lr": 1e-06
}

task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_energy],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4},
    scheduler_cls=ReduceLROnPlateau,
    scheduler_args=scheduler_args,
    scheduler_monitor="val_loss"
)

# create trainer
logger = pl.loggers.TensorBoardLogger(save_dir=workpath)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(workpath, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    accelerator='gpu',
    devices=devices,
    callbacks=callbacks,
    logger=logger,
    default_root_dir=workpath,
    max_epochs=nepochs,
)

lr_monitor = LearningRateMonitor(logging_interval='step')
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="step",
    mode="max",
    filename="last-chkpt-{epoch:02d}-{global_step}",
)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(workpath, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    ),
    lr_monitor,
    checkpoint_callback
]

trainer.fit(task, datamodule=data.dataset)

/mnt/2tb/rumiantsev/anaconda3/envs/check/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/lightning_logs
100%|███████████████████████████████████████████| 52/52 [00:01<00:00, 31.07it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 1.2 M 
1 | outputs | ModuleList             | 0     
---------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.628     Total estimated model param

Sanity Checking: 0it [00:00, ?it/s]

/mnt/2tb/rumiantsev/anaconda3/envs/check/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 100. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

/mnt/2tb/rumiantsev/anaconda3/envs/check/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:362: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `AtomisticTask.optimizer_step()` hook is overridden, including the `using_native_amp` argument. Removing this argument will avoid this message, you can expect it to return True.
  rank_zero_deprecation(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


## Initializing the testing procedure and computing the metric's result

In [5]:
dataset_name_test = 'dataset_test_conformations_2k'
datapath = 'database_test'
batch_size = 100
cutoff = 5.0
gpu = -1

if gpu == -1:
    device = torch.device("cpu")
else:
    device = torch.cuda.device(gpu)

device = torch.cuda.device(gpu)

data_test = NablaDFT(
    "ASE",
    dataset_name_test,
    datapath=datapath,
    data_workdir=workpath,
    distance_unit="Bohr",
    batch_size=batch_size,
    train_ratio=0,
    num_workers=1,
    transforms=[
        trn.ASENeighborList(cutoff=cutoff),
        trn.CastTo32()
    ],
    split_file=os.path.join(workpath, "split_test.npz")
)

data_test.dataset.prepare_data()
data_test.dataset.setup()

best_model = torch.load(os.path.join(workpath, 'best_inference_model'))
best_model = best_model.cuda()
best_model = best_model.eval()

metric = torchmetrics.MeanAbsoluteError()

with torch.no_grad():
    for x in tqdm.tqdm(data_test.dataset.val_dataloader()):
        for k in x:
            if x[k].dtype == torch.float64:
                x[k] = x[k].float()
            x[k] = x[k].to("cuda:0")

        target = x['energy'].cpu().clone()
        prediction = best_model(x)['energy'].cpu()
        mae = metric(prediction, target)

print(metric.compute())

100%|███████████████████████████████████████████| 18/18 [00:10<00:00,  1.69it/s]

tensor(0.7281)


## Downloading dataset with atomic energies subtracted and starting the training process.

In [ ]:
logspath = 'logs_remove_atomrefs'
workpath = logspath
data_remove_atomrefs = NablaDFT("ASE", dataset_name,
                datapath=datapath,
                data_workdir=workpath,
                batch_size=batch_size,
                num_workers=4,
                transforms=[
                            trn.ASENeighborList(cutoff=cutoff),
                            trn.RemoveOffsets("energy", remove_mean=True, remove_atomrefs=True),
                            trn.CastTo32()
                           ],
                split_file=os.path.join(workpath, "split.npz"))

pairwise_distance = spk.atomistic.PairwiseDistances()
radial_basis = spk.nn.radial.GaussianRBF(
    n_rbf=n_rbf,
    cutoff=cutoff
)
cutoff_fn = spk.nn.cutoff.CosineCutoff(cutoff)
representation = rep.PaiNN(
    n_interactions=n_interactions,
    n_atom_basis=n_atom_basis,
    radial_basis=radial_basis,
    cutoff_fn=cutoff_fn
)
pred_energy = spk.atomistic.Atomwise(
    n_in=representation.n_atom_basis,
    output_key="energy"
)
postprocessors = [
    trn.CastTo64(),
    trn.AddOffsets("energy", add_mean=True,
    add_atomrefs=True)
]
nnpot = spk.model.NeuralNetworkPotential(
    representation=representation,
    input_modules=[pairwise_distance],
    output_modules=[pred_energy],
    postprocessors=postprocessors
)
output_energy = spk.task.ModelOutput(
    name="energy",
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1,
    metrics={"MAE": torchmetrics.MeanAbsoluteError()}
)

scheduler_args = {
    "factor": 0.8,
    "patience": 10,
    "min_lr": 1e-06
}

task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_energy],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4},
    scheduler_cls=ReduceLROnPlateau,
    scheduler_args=scheduler_args,
    scheduler_monitor="val_loss"
)

# create trainer
logger = pl.loggers.TensorBoardLogger(save_dir=workpath)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(workpath, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    accelerator='gpu',
    devices=devices,
    callbacks=callbacks,
    logger=logger,
    default_root_dir=workpath,
    max_epochs=nepochs,
)

lr_monitor = LearningRateMonitor(logging_interval='step')
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="step",
    mode="max",
    filename="last-chkpt-{epoch:02d}-{global_step}",
)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(workpath, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    ),
    lr_monitor,
    checkpoint_callback
]

trainer.fit(task, datamodule=data_remove_atomrefs.dataset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs_remove_atomrefs/lightning_logs
100%|███████████████████████████████████████████| 52/52 [00:02<00:00, 25.99it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 1.2 M 
1 | outputs | ModuleList             | 0     
---------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.628     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [ ]:
datapath = 'database_test'
batch_size = 100
cutoff = 5.0
gpu = -1

if gpu == -1:
    device = torch.device("cpu")
else:
    device = torch.cuda.device(gpu)

device = torch.cuda.device(gpu)

data_remove_atomrefs_test = NablaDFT(
    "ASE",
    dataset_name_test,
    datapath=datapath,
    data_workdir=workpath,
    distance_unit="Bohr",
    batch_size=batch_size,
    train_ratio=0,
    num_workers=1,
    transforms=[
        trn.ASENeighborList(cutoff=cutoff),
        trn.CastTo32()
    ],
    split_file=os.path.join(workpath, "split_test.npz")
)

data_remove_atomrefs_test.dataset.prepare_data()
data_remove_atomrefs_test.dataset.setup()

best_model = torch.load(os.path.join(workpath, 'best_inference_model'))
best_model = best_model.cuda()
best_model = best_model.eval()

metric = torchmetrics.MeanAbsoluteError()

with torch.no_grad():
    for x in tqdm.tqdm(data_remove_atomrefs_test.dataset.val_dataloader()):
        for k in x:
            if x[k].dtype == torch.float64:
                x[k] = x[k].float()
            x[k] = x[k].to("cuda:0")

        target = x['energy'].cpu().clone()
        prediction = best_model(x)['energy'].cpu()
        mae = metric(prediction, target)

print(metric.compute())

100%|███████████████████████████████████████████| 18/18 [00:10<00:00,  1.67it/s]

tensor(0.0132)
